In [1]:
print('aaa')

In [ ]:
def _gen_training_samples(n, start_i):
    X = np.zeros((n, ) + INPUT_SIZE, dtype=np.float32)
    Y = np.zeros((n, len(LABELS)), dtype=np.float32)
    for i in range(n):
        wave, label = dg.generate_audio()
        msg = dg.normalize_msg(dg.msg(wave))
        msg = np.expand_dims(msg, 2)
        X[i] = msg
        Y[i] = dg.label_to_onehot(label)
    np.save('%s/X_%07d-%07d' % (TRAIN_TMP_DIR, start_i, n + start_i), X)
    np.save('%s/Y_%07d-%07d' % (TRAIN_TMP_DIR, start_i, n + start_i), Y)


def generate_train_set(n_total=100, n_per_job=10, n_pools=16):

    global train_X
    global train_Y

    assert n_total % n_per_job == 0

    # cleanup tmp dir
    if os.path.isdir(TRAIN_TMP_DIR): shutil.rmtree(TRAIN_TMP_DIR)

    # create temporary dir for generated data
    if not os.path.isdir(TRAIN_TMP_DIR): os.makedirs(TRAIN_TMP_DIR)

    # launch generation in pool of workers

    n_jobs = n_total // n_per_job
    params = map(lambda x: [n_per_job, x * n_per_job], range(n_jobs))

    with Pool(n_pools) as p:
        p.starmap(_gen_training_samples, list(params))

    # glue generated files together

    train_X = np.zeros((n_total, ) + INPUT_SIZE, dtype=np.float32)
    train_Y = np.zeros((n_total, len(dg.labels)), dtype=np.float32)

    for i in range(0, n_total, n_per_job):
        X_file = '%s/X_%07d-%07d.npy' % (TRAIN_TMP_DIR, i, i + n_per_job)
        Y_file = '%s/Y_%07d-%07d.npy' % (TRAIN_TMP_DIR, i, i + n_per_job)
        X = np.load(X_file)
        Y = np.load(Y_file)
        train_X[i:i + n_per_job] = X
        train_Y[i:i + n_per_job] = Y

    np.save(train_X_file, train_X)
    np.save(train_Y_file, train_Y)